# 提案手法の実験（ラベルが適切か出力）

## マルチサイズ
- データオーギュメンテーション（鏡映，回転を追加）

### import，入力データの読み込み

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('../input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

../input/LSWMD.pkl


In [2]:
import os
from os.path import join
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

import csv

import numpy as np
import pandas as pd
import pickle
import copy
import cv2
import random
import glob

from sklearn.model_selection import KFold 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import joblib

import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)
if len(physical_devices) > 0:
    for device in physical_devices:
        tf.config.experimental.set_memory_growth(device, True)
        print('{} memory growth: {}'.format(device, tf.config.experimental.get_memory_growth(device)))
else:
    print("Not enough GPU hardware devices available")
logical_gpus = tf.config.experimental.list_logical_devices('GPU')
print(logical_gpus)
import keras
from tensorflow.keras import layers, Input, models
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier 
import keras.backend.tensorflow_backend as tfback
# from tf.keras.utils import multi_gpu_model


import matplotlib.pyplot as plt

from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

datapath = join('data', 'wafer')
print(os.listdir("../input"))
import warnings
warnings.filterwarnings("ignore")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]
PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU') memory growth: True
PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU') memory growth: True
[LogicalDevice(name='/device:GPU:0', device_type='GPU'), LogicalDevice(name='/device:GPU:1', device_type='GPU')]


Using TensorFlow backend.


['LSWMD.pkl']


In [3]:
# Define
max_size = 300
encord_size = int(max_size / 2)

MAKE_DATASET = False
TRAIN_AUTO_ENCODER = False

cnn_path = './model/cnn_' + str(max_size) + '.h5'

epoch = 30
batch_size = 2048

In [4]:
faulty_case = ['Center', 'Donut', 'Edge-Loc', 'Edge-Ring', 'Loc', 'Near-full', 'Random', 'Scratch', 'none']
print('Faulty case list : {}'.format(faulty_case))

Faulty case list : ['Center', 'Donut', 'Edge-Loc', 'Edge-Ring', 'Loc', 'Near-full', 'Random', 'Scratch', 'none']


### 各ウエハにラベル付け

In [5]:
# trials = 2
# label_list = []
# for i in range(new_x.shape[0]):
#     label_dict = {'wafer_id':str(i).zfill(6), 'true_label':y[i][0], 'predict_label':None, 'augmentation':{'noise':0, 'rotation':0, 'inversion':0}, 'trials':trials}
#     label_list.append(label_dict)

In [6]:
# print(label_list[1])

- 不良ラベルが付いているデータに対してデータオーギュメンテーションを行う．

### 学習を行う
- 不良ラベルを0-8の9次元のベクトルとして表現する．
- one-hotエンコーディングを行っている．

- 保存/読み込み

In [7]:
# MAKE_DATASET = False
# ver = 'shawon' if shawon else 'propose'
# if MAKE_DATASET:
#     pickle_dump(new_x, './data/new_x_' + ver + '.pickle')
#     pickle_dump(new_y, './data/new_y_' + ver + '.pickle')
#     pickle_dump(label_list, './data/label_list_' + ver + '.pickle')
    
# if not MAKE_DATASET:
#     new_x = pickle_load('./data/new_x_' + ver + '.pickle')
#     new_y = pickle_load('./data/new_y_' + ver + '.pickle')
#     label_list = pickle_load('./data/label_list_' + ver + '.pickle')

In [8]:
# for i, l in enumerate(faulty_case):
#     new_y[new_y==l] = i

In [9]:
# for i in range(len(label_list)):
#     label_list[i]['true_label'] = new_y[i][0]

### データの読み出し

In [10]:
# acquire the .npy name
data_size = len(glob.glob('./data/multi_' + str(max_size) + '/train/' + '*.npy'))
TRAINS = ['./data/multi_' + str(max_size) + '/train/' + str(i) + '.npy' for i in range(data_size)]
# one-hot-encoding
y = joblib.load('./data/multi_' + str(max_size) + '/train/y.pickle')
new_y = to_categorical(y)
# split test

# shuffle_indices = random.sample(list(range(len(TRAINS))), 10000)
# TRAINS = [TRAINS[i] for i in shuffle_indices]
# new_y = new_y[shuffle_indices]

indices = np.array(range(len(TRAINS)))
x_train, x_validation, y_train, y_validation, indices_train, indices_validation = train_test_split(
    TRAINS, new_y, indices, test_size=0.01, random_state=2020)

In [11]:
# batchを取得する関数
from multiprocessing import Pool
import time

def load_array(file):
    return np.load(file)

def get_batch(batch_size): 
    global x_train, y_train
    SIZE = len(x_train)
    # n_batchs
    n_batchs = SIZE//batch_size + 1
    # for でyield
    i = 0
    start = time.time()
    while (i < n_batchs):
        print("doing", i, "/", n_batchs)
        Y_batch = y_train[(i * batch_size):((i + 1) * batch_size)]
        
        #あるbatchのfilenameの配列を持っておく
        X_batch_name = x_train[(i * batch_size):((i + 1) * batch_size)]

        # filenameにしたがってバッチのtensorを構築
        with Pool() as p:
            arr = p.map(load_array, X_batch_name)
            
        X_batch = np.array(arr).reshape(len(X_batch_name), max_size, max_size, 3)
#         X_batch = np.array([np.load(file)
#                             for file in X_batch_name]).reshape(len(X_batch_name), max_size, max_size, 3)
        i += 1
        print('elapsed time', time.time()-start)
        yield X_batch, Y_batch

In [12]:
# # testsize = 2000
# # randlist = rand_ints_nodup(0, new_x.shape[0]-1, testsize)

# # valx = new_x.copy()[randlist, :, :, :]
# # valy = y.copy()[randlist, :]

# test_size = 705 #705
# new_x_size = new_x.shape[0]
# testlist = rand_ints_nodup(0, new_x_size-1, test_size)
# trainlist = [i for i in range(new_x_size) if i != testlist]
# new_X=new_x[trainlist]
# new_Y=new_y[trainlist]
# test_x=new_x[testlist]
# test_y=new_y[testlist]

# label_train = copy.deepcopy([label_list[i] for i in trainlist])
# label_test = copy.deepcopy([label_list[i] for i in testlist])

# test_x.shape

In [13]:
# new_X=new_x
# new_Y=new_y

- 学習データとテストデータに分割する．

In [14]:
# indices = np.array(range(new_X.shape[0]))

# x_train, x_test, y_train, y_test, indices_train, indices_test = train_test_split(new_X, new_Y, indices,
#                                                                 test_size=0.33,
#                                                                 random_state=2020)

In [15]:
# label_vali = copy.deepcopy([label_train[i] for i in indices_test.tolist()])
# label_train = copy.deepcopy([label_train[i] for i in indices_train.tolist()])

In [16]:
print('Train x : {}, y : {}'.format(len(x_train), y_train.shape))
print('Validation x: {}, y : {}'.format(len(x_validation), y_validation.shape))

Train x : 299014, y : (299014, 9)
Validation x: 3021, y : (3021, 9)


In [17]:
print("loading X_validation...")
with Pool() as p:
    arr = p.map(load_array, x_validation)

x_validation = np.array(arr).reshape(len(x_validation), max_size, max_size, 3)

loading X_validation...


- 学習データ246635枚，テストデータ121477枚．

- モデルの定義を行う．

### CNN

In [18]:
def create_model():
    with tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"], 
                                        cross_device_ops = tf.distribute.HierarchicalCopyAllReduce()).scope():
        input_shape = (max_size, max_size, 3)
        input_tensor = Input(input_shape)

        conv_1 = layers.Conv2D(8, (3,3), activation='relu', padding='same')(input_tensor)
        conv_2 = layers.Conv2D(16, (3,3), activation='relu', padding='same')(conv_1)
        conv_3 = layers.Conv2D(32, (3,3), activation='relu', padding='same')(conv_2)

        flat = layers.Flatten()(conv_3)

        dense_1 = layers.Dense(64, activation='relu')(flat)
        dense_2 = layers.Dense(32, activation='relu')(dense_1)
        output_tensor = layers.Dense(9, activation='softmax')(dense_2)

        model = models.Model(input_tensor, output_tensor)
        model.compile(optimizer='Adam',
                     loss='categorical_crossentropy',
                     metrics=['accuracy'])

    return model

- 3-Fold Cross validationで分割して学習する．

In [19]:
# model = KerasClassifier(build_fn=create_model, epochs=30, batch_size=1024, verbose=1) 
# # 3-Fold Crossvalidation
# kfold = KFold(n_splits=3, shuffle=True, random_state=2019) 
# # results = cross_val_score(model, x_train, y_train, cv=kfold)
# # # Check 3-fold model's mean accuracy
# # print('Simple CNN Cross validation score : {:.4f}'.format(np.mean(results)))

- Cross validiationによる精度は99.55%であった．

- Cross validationなしで学習する．

In [20]:
strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1", "device:XLA_GPU:0"], cross_device_ops = tf.distribute.HierarchicalCopyAllReduce())
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:XLA_GPU:0')
Number of devices: 3


In [21]:
epoch=5
batch_size=256
random.seed(1)

In [ ]:
model = create_model()
epoch_train_loss = []
epoch_train_acc = []
epoch_validation_loss = []
epoch_validation_acc = []
for ep in range(epoch):
    print("=" * 50)
    print(ep, "/", epoch)
    step_loss = []
    step_acc = []
    
    # batch_size=1000でHDDからバッチを取得する
    for X_batch, Y_batch in get_batch(batch_size):
        model.train_on_batch(X_batch, Y_batch)
        score = model.evaluate(X_batch, Y_batch)
        print("batch loss:", score[0])
        print("batch accuracy:", score[1])
        step_loss.append(score[0])
        step_acc.append(score[1])
    print("Train loss", np.mean(step_loss))
    print("Train accuracy", np.mean(step_acc))
    score = model.evaluate(x_validation, y_validation)
    print("Validation loss:", score[0])
    print("Validation accuracy:", score[1])
    epoch_train_loss.append(np.mean(step_loss))
    epoch_train_acc.append(np.mean(step_acc))
    epoch_validation_loss.append(score[0])
    epoch_validation_acc.append(score[1])
    
    shuffle_indices = random.sample(list(range(len(x_train))), len(x_train))
    x_train = [x_train[i] for i in shuffle_indices]
    y_train = y_train[shuffle_indices]

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
0 / 5
doing 0 / 1169
elapsed time 0.40367674827575684
INFO:tensorflow:batch_all_reduce: 12 all-reduces with algorithm = hierarchical_copy, num_packs = 1
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:batch_all_reduce: 12 all-reduces with algorithm = hierarchical_copy, num_packs = 1
INFO:tensorflow:Reduce to /job:localhos

8/8 [==============================] - 0s 16ms/step - loss: 2.1809 - accuracy: 0.1484
batch loss: 2.180910587310791
batch accuracy: 0.1484375
doing 34 / 1169
elapsed time 865.9089155197144
8/8 [==============================] - 0s 16ms/step - loss: 2.1875 - accuracy: 0.1133
batch loss: 2.187547206878662
batch accuracy: 0.11328125
doing 35 / 1169
elapsed time 891.3984458446503
8/8 [==============================] - 0s 15ms/step - loss: 2.1814 - accuracy: 0.1055
batch loss: 2.1814332008361816
batch accuracy: 0.10546875
doing 36 / 1169
elapsed time 916.0800135135651
8/8 [==============================] - 0s 16ms/step - loss: 2.1567 - accuracy: 0.1523
batch loss: 2.1566858291625977
batch accuracy: 0.15234375
doing 37 / 1169
elapsed time 941.3701982498169
8/8 [==============================] - 0s 16ms/step - loss: 2.1688 - accuracy: 0.1641
batch loss: 2.1687872409820557
batch accuracy: 0.1640625
doing 38 / 1169
elapsed time 965.8117790222168
8/8 [==============================] - 0s 15ms/st

8/8 [==============================] - 0s 15ms/step - loss: 1.9392 - accuracy: 0.3398
batch loss: 1.9392063617706299
batch accuracy: 0.33984375
doing 77 / 1169
elapsed time 2164.1038222312927
8/8 [==============================] - 0s 15ms/step - loss: 1.9800 - accuracy: 0.2734
batch loss: 1.9800477027893066
batch accuracy: 0.2734375
doing 78 / 1169
elapsed time 2188.6889917850494
8/8 [==============================] - 0s 16ms/step - loss: 1.9013 - accuracy: 0.3477
batch loss: 1.901323914527893
batch accuracy: 0.34765625
doing 79 / 1169
elapsed time 2213.3272585868835
8/8 [==============================] - 0s 15ms/step - loss: 1.9253 - accuracy: 0.3477
batch loss: 1.9252727031707764
batch accuracy: 0.34765625
doing 80 / 1169
elapsed time 2237.8619182109833
8/8 [==============================] - 0s 16ms/step - loss: 1.9521 - accuracy: 0.2695
batch loss: 1.9521002769470215
batch accuracy: 0.26953125
doing 81 / 1169
elapsed time 2263.1764488220215
8/8 [==============================] - 0s 

8/8 [==============================] - 0s 16ms/step - loss: 1.7803 - accuracy: 0.3984
batch loss: 1.7803099155426025
batch accuracy: 0.3984375
doing 120 / 1169
elapsed time 2919.517336368561
8/8 [==============================] - 0s 16ms/step - loss: 1.4709 - accuracy: 0.4570
batch loss: 1.470863699913025
batch accuracy: 0.45703125
doing 121 / 1169
elapsed time 2927.737204313278
8/8 [==============================] - 0s 15ms/step - loss: 1.5330 - accuracy: 0.4414
batch loss: 1.53303861618042
batch accuracy: 0.44140625
doing 122 / 1169
elapsed time 2936.272951364517
8/8 [==============================] - 0s 16ms/step - loss: 1.4964 - accuracy: 0.4688
batch loss: 1.4964052438735962
batch accuracy: 0.46875
doing 123 / 1169
elapsed time 2944.5523784160614
8/8 [==============================] - 0s 16ms/step - loss: 1.6760 - accuracy: 0.3867
batch loss: 1.6759634017944336
batch accuracy: 0.38671875
doing 124 / 1169
elapsed time 2953.590071439743
8/8 [==============================] - 0s 16ms

elapsed time 3296.370346546173
8/8 [==============================] - 0s 15ms/step - loss: 1.4950 - accuracy: 0.4766
batch loss: 1.4950425624847412
batch accuracy: 0.4765625
doing 163 / 1169
elapsed time 3305.4701359272003
8/8 [==============================] - 0s 15ms/step - loss: 1.5633 - accuracy: 0.4883
batch loss: 1.5633044242858887
batch accuracy: 0.48828125
doing 164 / 1169
elapsed time 3314.37153840065
8/8 [==============================] - 0s 16ms/step - loss: 1.5586 - accuracy: 0.4414
batch loss: 1.5585546493530273
batch accuracy: 0.44140625
doing 165 / 1169
elapsed time 3323.3207347393036
8/8 [==============================] - 0s 16ms/step - loss: 1.4132 - accuracy: 0.5352
batch loss: 1.4131584167480469
batch accuracy: 0.53515625
doing 166 / 1169
elapsed time 3332.1881098747253
8/8 [==============================] - 0s 16ms/step - loss: 1.4402 - accuracy: 0.4922
batch loss: 1.440242052078247
batch accuracy: 0.4921875
doing 167 / 1169
elapsed time 3342.032644033432
8/8 [=====

elapsed time 3677.085534095764
8/8 [==============================] - 0s 15ms/step - loss: 1.3005 - accuracy: 0.5586
batch loss: 1.3004775047302246
batch accuracy: 0.55859375
doing 206 / 1169
elapsed time 3687.020094871521
8/8 [==============================] - 0s 15ms/step - loss: 1.2766 - accuracy: 0.5664
batch loss: 1.2766220569610596
batch accuracy: 0.56640625
doing 207 / 1169
elapsed time 3696.0073478221893
8/8 [==============================] - 0s 16ms/step - loss: 1.3338 - accuracy: 0.5820
batch loss: 1.333793044090271
batch accuracy: 0.58203125
doing 208 / 1169
elapsed time 3704.634191274643
8/8 [==============================] - 0s 15ms/step - loss: 1.3957 - accuracy: 0.5312
batch loss: 1.3956512212753296
batch accuracy: 0.53125
doing 209 / 1169
elapsed time 3713.1375319957733
8/8 [==============================] - 0s 16ms/step - loss: 1.2657 - accuracy: 0.6133
batch loss: 1.2657263278961182
batch accuracy: 0.61328125
doing 210 / 1169
elapsed time 3721.3248727321625
8/8 [=====

elapsed time 4049.424317598343
8/8 [==============================] - 0s 15ms/step - loss: 1.1419 - accuracy: 0.6289
batch loss: 1.1418870687484741
batch accuracy: 0.62890625
doing 249 / 1169
elapsed time 4057.6547949314117
8/8 [==============================] - 0s 16ms/step - loss: 1.1410 - accuracy: 0.6328
batch loss: 1.1409518718719482
batch accuracy: 0.6328125
doing 250 / 1169
elapsed time 4065.9125785827637
8/8 [==============================] - 0s 15ms/step - loss: 1.1714 - accuracy: 0.6016
batch loss: 1.171410322189331
batch accuracy: 0.6015625
doing 251 / 1169
elapsed time 4074.0183877944946
8/8 [==============================] - 0s 16ms/step - loss: 1.2006 - accuracy: 0.5547
batch loss: 1.2005503177642822
batch accuracy: 0.5546875
doing 252 / 1169
elapsed time 4082.042383670807
8/8 [==============================] - 0s 15ms/step - loss: 1.0272 - accuracy: 0.6172
batch loss: 1.0271589756011963
batch accuracy: 0.6171875
doing 253 / 1169
elapsed time 4091.8052928447723
8/8 [=====

In [ ]:
# model.save(cnn_path)

In [ ]:
import sys

print("{}{: >25}{}{: >10}{}".format('|','Variable Name','|','Memory','|'))
print(" ------------------------------------ ")
for var_name in dir():
    if not var_name.startswith("_"):
        print("{}{: >25}{}{: >10}{}".format('|',var_name,'|',sys.getsizeof(eval(var_name)),'|'))

### テストデータで評価

In [ ]:
x_test = joblib.load('./data/multi_' + str(max_size) + '/test/xtest.pickle')
y_test = joblib.load('./data/multi_' + str(max_size) + '/test/ytest.pickle')
y_test = to_categorical(y_test)

In [ ]:
score = model.evaluate(x_test, y_test)
print("Test loss:", score[0])
print("Test accuracy:", score[1])
testscore = score[1]
trainscore = epoch_train_acc[-1]
valiscore = epoch_validation_acc[-1]
print("Train accuracy:", trainscore)
print("Validation accuracy:", valiscore)
print("Test accuracy:", testscore)

In [ ]:
y_predict = np.argmax(model.predict(x_test), axis=1)
y_test_max = np.argmax(y_test, axis=1)
np.sum(y_test_max == y_predict, axis=0, dtype='float') / x_test.shape[0]

- モデルは以下．
    - 入力層
    - 畳み込み層3つ
    - Flatten層（1次元に）
    - 全結合層3つ

In [ ]:
model = create_model()
model.summary()

- accuracyグラフ，lossグラフは以下．
- 5epoch程度で落ち着いている．

In [ ]:
# accuracy plot
fig1 = plt.figure()
plt.plot(epoch_train_acc)
plt.plot(epoch_validation_acc)
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
fig1.savefig("multisize_accuracy.png")

# loss plot
fig2 = plt.figure()
plt.plot(epoch_train_loss)
plt.plot(epoch_validation_loss)
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
fig2.savefig("multisize_loss.png")

### 混同行列

In [ ]:
import itertools
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(cm, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    #print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    fmt = '.4f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    
    plt.xticks([0, 1, 2, 3, 4, 5, 6, 7 ,8], ["Center", "Donut", "Edge-Loc", "Edge-Ring", "Loc", "Near-full", "Random", "Scratch", "None"])
    plt.yticks([0, 1, 2, 3, 4, 5, 6, 7 ,8], ["Center", "Donut", "Edge-Loc", "Edge-Ring", "Loc", "Near-full", "Random", "Scratch", "None"])
    
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

- validation confmat

In [ ]:
# Compute confusion matrix
y_validation_predict = np.argmax(model.predict(x_validation), axis=1)
y_validation_max = np.argmax(y_validation, axis=1)
cnf_matrix = confusion_matrix(y_validation_max, y_validation_predict)
np.set_printoptions(precision=2)

from matplotlib import gridspec
fig = plt.figure(figsize=(8, 15)) 
gs = gridspec.GridSpec(2, 1, height_ratios=[1, 1]) 

## Plot non-normalized confusion matrix
plt.subplot(gs[0])
plot_confusion_matrix(cnf_matrix, title='Confusion matrix')

# Plot normalized confusion matrix
plt.subplot(gs[1])
plot_confusion_matrix(cnf_matrix, normalize=True, title='Normalized confusion matrix')

plt.show()
fig.savefig("multisize_valiconfmat.png")

- test confmat

In [ ]:
# Compute confusion matrix
y_test_predict = np.argmax(model.predict(x_test), axis=1)
y_test_max = np.argmax(y_test, axis=1)
cnf_matrix = confusion_matrix(y_test_max, y_test_predict)
np.set_printoptions(precision=2)

from matplotlib import gridspec
fig = plt.figure(figsize=(8, 15)) 
gs = gridspec.GridSpec(2, 1, height_ratios=[1, 1]) 

## Plot non-normalized confusion matrix
plt.subplot(gs[0])
plot_confusion_matrix(cnf_matrix, title='Confusion matrix')

# Plot normalized confusion matrix
plt.subplot(gs[1])
plot_confusion_matrix(cnf_matrix, normalize=True, title='Normalized confusion matrix')

plt.show()
fig.savefig("multisize_testconfmat.png")

In [ ]:
%%javascript
var notebook_name = document.body.attributes['data-notebook-name'].value
IPython.notebook.kernel.execute(`NOTEBOOK_NAME = '${notebook_name}'`);

In [ ]:
import requests

# LINEの設定
path = './lineapi.txt'
with open(path) as f:
    s = f.read()
    line_token = s.rstrip('\n')

# LINEに通知する関数
def line_notify(text):
    url = "https://notify-api.line.me/api/notify"
    data = {"message": text}
    headers = {"Authorization": "Bearer " + line_token}
    proxies = {
        'http': 'http://proxy.uec.ac.jp:8080',
        'https': 'https://proxy.uec.ac.jp:8080',
    }
    requests.post(url, data=data, headers=headers)#, proxies=proxies)

# LINEに画像を送る関数
def line_notify_img(text, imgpath):
    url = "https://notify-api.line.me/api/notify"
    data = {"message": text, "notificationDisabled": True}
    files = {"imageFile": open(imgpath, "rb")}
    headers = {"Authorization": "Bearer " + line_token}
    proxies = {
        'http': 'http://proxy.uec.ac.jp:8080',
        'https': 'https://proxy.uec.ac.jp:8080',
    }
    requests.post(url, data=data, files=files, headers=headers)#, proxies=proxies)
    
line_notify("学習が終了しました " + NOTEBOOK_NAME)
# line_notify("Shawon: " + str(shawon) + ", rotation_num: " + str(rotation_num) + ", inversion: " + str(inversion) + ", trials: " + str(trials))
line_notify_img("正解率", "multisize_accuracy.png")
line_notify_img("Loss", "multisize_loss.png")
line_notify_img("validation混同行列", "multisize_valiconfmat.png")
line_notify_img("test混同行列", "multisize_testconfmat.png")
line_notify("train:" + str(trainscore) + "\nvali:" + str(valiscore) + "\ntest:" + str(testscore))